In [ ]:
from timeit import default_timer
import numpy as np

from OverheadTester import OverheadedTester

def Solid(image, kernel):
    nRows = image.shape[0]
    nCols = image.shape[1]

    nRows_kernel = kernel.shape[0]
    nCols_kernel = kernel.shape[1]

    center_r = (nRows_kernel-1) // 2
    center_c = (nCols_kernel-1) // 2

    acc = 0.0

    conv_out = np.zeros((nRows, nCols), dtype=np.float32)

    for r in range(nRows):
        for c in range(nCols):
            acc = 0
            for kr in range(nRows_kernel):
                for kc in range(nCols_kernel):
                    local_row = min(max(r+(kr-center_r), 0), nRows-1)
                    local_col = min(max(c+(kc-center_c), 0), nCols-1)
                    acc = acc + kernel[kr, kc] * image[local_row, local_col]
            conv_out[r, c] = acc

    return conv_out

ot = OverheadedTester()

img = np.random.random((300,300)).astype(np.float32)
kn = np.random.random((10,10)).astype(np.float32)

In [ ]:
for _ in range(3):
    ot.benchmark(img,kn)

In [ ]:
t_liquidagent = []
for _ in range(20):

    t1 = default_timer()
    ot.run(img,kn)
    t_liquidagent.append(default_timer()-t1)

t_liquidpython = []
for _ in range(20):

    t1 = default_timer()
    ot.run(img,kn,run_type="Python")
    t_liquidpython.append(default_timer()-t1)

t_solid = []
for _ in range(20):

    t1 = default_timer()
    Solid(img,kn)
    t_solid.append(default_timer()-t1)

In [ ]:
print(np.average(t_liquidagent)-np.average(t_liquidpython))

print(np.average(t_liquidagent)-np.average(t_solid))

print(np.average(t_liquidpython)-np.average(t_solid))
